In [718]:
import numpy as np
from keras.layers import TimeDistributed
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.layers.recurrent import GRU,LSTM
import pandas as pd
from collections import defaultdict
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
import csv
import tensorflow as tf

In [872]:
csv = pd.read_csv("/Users/tanaykarmarkar/Downloads/HistoricalQuotes.csv" , sep=',')

In [873]:
csv

,Date,Open,High,Low,Close,Volume
0,13-Oct-17,77.59,77.87,77.29,77.49,15335742
1,12-Oct-17,76.49,77.29,76.37,77.12,16876538
2,11-Oct-17,76.36,76.46,75.95,76.42,15388898
3,10-Oct-17,76.33,76.63,76.14,76.29,13944545
4,9-Oct-17,75.97,76.55,75.86,76.29,11386502
5,6-Oct-17,75.67,76.03,75.54,76.00,13959814
6,5-Oct-17,75.22,76.12,74.96,75.97,21195261
7,4-Oct-17,74.09,74.72,73.71,74.69,13317681
8,3-Oct-17,74.67,74.88,74.20,74.26,12190403
9,2-Oct-17,74.71,75.01,74.30,74.61,15304762


In [875]:
target=csv[['Open']].values

In [876]:
target

array([[ 77.59],
       [ 76.49],
       [ 76.36],
       [ 76.33],
       [ 75.97],
       [ 75.67],
       [ 75.22],
       [ 74.09],
       [ 74.67],
       [ 74.71],
       [ 73.94],
       [ 73.54],
       [ 73.55],
       [ 73.67],
       [ 74.09],
       [ 73.99],
       [ 75.11],
       [ 75.35],
       [ 75.21],
       [ 75.23],
       [ 74.83],
       [ 75.  ],
       [ 74.93],
       [ 74.76],
       [ 74.31],
       [ 74.33],
       [ 73.68],
       [ 73.74],
       [ 73.34],
       [ 74.71],
       [ 74.03],
       [ 73.01],
       [ 72.25],
       [ 73.06],
       [ 72.86],
       [ 72.74],
       [ 72.96],
       [ 72.35],
       [ 72.47],
       [ 72.27],
       [ 73.58],
       [ 73.34],
       [ 73.59],
       [ 73.06],
       [ 71.61],
       [ 71.9 ],
       [ 72.25],
       [ 72.09],
       [ 72.8 ],
       [ 72.4 ],
       [ 72.19],
       [ 72.55],
       [ 73.1 ],
       [ 73.3 ],
       [ 72.67],
       [ 73.76],
       [ 74.34],
       [ 73.8 ],
       [ 73.53

In [877]:
data=csv[['Date']].values

In [878]:
data

array([['13-Oct-17'],
       ['12-Oct-17'],
       ['11-Oct-17'],
       ['10-Oct-17'],
       ['9-Oct-17'],
       ['6-Oct-17'],
       ['5-Oct-17'],
       ['4-Oct-17'],
       ['3-Oct-17'],
       ['2-Oct-17'],
       ['29-Sep-17'],
       ['28-Sep-17'],
       ['27-Sep-17'],
       ['26-Sep-17'],
       ['25-Sep-17'],
       ['22-Sep-17'],
       ['21-Sep-17'],
       ['20-Sep-17'],
       ['19-Sep-17'],
       ['18-Sep-17'],
       ['15-Sep-17'],
       ['14-Sep-17'],
       ['13-Sep-17'],
       ['12-Sep-17'],
       ['11-Sep-17'],
       ['8-Sep-17'],
       ['7-Sep-17'],
       ['6-Sep-17'],
       ['5-Sep-17'],
       ['1-Sep-17'],
       ['31-Aug-17'],
       ['30-Aug-17'],
       ['29-Aug-17'],
       ['28-Aug-17'],
       ['25-Aug-17'],
       ['24-Aug-17'],
       ['23-Aug-17'],
       ['22-Aug-17'],
       ['21-Aug-17'],
       ['18-Aug-17'],
       ['17-Aug-17'],
       ['16-Aug-17'],
       ['15-Aug-17'],
       ['14-Aug-17'],
       ['11-Aug-17'],
       ['10-Aug-17'],

In [879]:
len(data)

251

In [880]:
index = np.arange(251)

In [881]:
index2date = dict(zip(index,data))

In [882]:
index2date

{0: array(['13-Oct-17'], dtype=object),
 1: array(['12-Oct-17'], dtype=object),
 2: array(['11-Oct-17'], dtype=object),
 3: array(['10-Oct-17'], dtype=object),
 4: array(['9-Oct-17'], dtype=object),
 5: array(['6-Oct-17'], dtype=object),
 6: array(['5-Oct-17'], dtype=object),
 7: array(['4-Oct-17'], dtype=object),
 8: array(['3-Oct-17'], dtype=object),
 9: array(['2-Oct-17'], dtype=object),
 10: array(['29-Sep-17'], dtype=object),
 11: array(['28-Sep-17'], dtype=object),
 12: array(['27-Sep-17'], dtype=object),
 13: array(['26-Sep-17'], dtype=object),
 14: array(['25-Sep-17'], dtype=object),
 15: array(['22-Sep-17'], dtype=object),
 16: array(['21-Sep-17'], dtype=object),
 17: array(['20-Sep-17'], dtype=object),
 18: array(['19-Sep-17'], dtype=object),
 19: array(['18-Sep-17'], dtype=object),
 20: array(['15-Sep-17'], dtype=object),
 21: array(['14-Sep-17'], dtype=object),
 22: array(['13-Sep-17'], dtype=object),
 23: array(['12-Sep-17'], dtype=object),
 24: array(['11-Sep-17'], dtype=

In [892]:
indextrain, indextest, targettrain, targettest = train_test_split(index , target , test_size=0.22)

In [895]:
indextrain1=indextrain.reshape(65,1,3)

In [897]:
indextrain1

array([[[ 81, 249, 170]],

       [[204,  23, 167]],

       [[ 15, 127,  76]],

       [[186,   3,  36]],

       [[ 41, 120, 130]],

       [[  5,   7,   8]],

       [[163, 159,  68]],

       [[ 11, 189, 100]],

       [[116, 208, 102]],

       [[ 22,  65,  83]],

       [[132, 164, 210]],

       [[150, 182,  39]],

       [[ 53,  34, 181]],

       [[ 89, 162,  18]],

       [[ 84, 222,  79]],

       [[122, 229, 109]],

       [[133,  40, 131]],

       [[185, 233,  57]],

       [[146, 202,  48]],

       [[234, 242,  27]],

       [[108, 166, 168]],

       [[165, 171, 177]],

       [[160, 237, 110]],

       [[ 17, 176, 247]],

       [[139,  67, 213]],

       [[243,  38, 128]],

       [[ 55,  62,  29]],

       [[ 64,  90, 235]],

       [[  2, 135,  92]],

       [[188, 194,  19]],

       [[200,  98,  59]],

       [[241, 180, 155]],

       [[179,  80, 248]],

       [[129,   0, 145]],

       [[ 10, 216,  60]],

       [[238,  78, 142]],

       [[ 96, 154, 245]],

 

In [898]:
targettrain1=targettrain.reshape(65,1,3)

In [899]:
targettrain1

array([[[ 70.82,  57.53,  64.25]],

       [[ 63.43,  74.76,  64.5 ]],

       [[ 73.99,  65.29,  70.11]],

       [[ 62.24,  76.33,  72.96]],

       [[ 73.34,  67.9 ,  65.61]],

       [[ 75.67,  74.09,  74.67]],

       [[ 64.33,  64.08,  69.46]],

       [[ 73.54,  62.62,  68.72]],

       [[ 68.68,  62.7 ,  67.5 ]],

       [[ 74.93,  71.5 ,  69.73]],

       [[ 65.6 ,  64.61,  62.5 ]],

       [[ 65.01,  63.95,  72.27]],

       [[ 73.3 ,  72.86,  64.12]],

       [[ 72.51,  64.42,  75.21]],

       [[ 69.27,  60.3 ,  70.54]],

       [[ 65.67,  58.33,  68.99]],

       [[ 66.3 ,  73.58,  65.85]],

       [[ 62.67,  60.  ,  73.8 ]],

       [[ 64.91,  63.45,  72.8 ]],

       [[ 60.55,  60.61,  73.74]],

       [[ 68.36,  64.74,  64.41]],

       [[ 64.47,  63.52,  64.36]],

       [[ 64.54,  59.53,  68.86]],

       [[ 75.35,  63.25,  57.5 ]],

       [[ 64.96,  70.11,  61.3 ]],

       [[ 60.81,  72.47,  65.42]],

       [[ 73.76,  73.09,  74.71]],

       [[ 72.24,  72.64,  59

In [914]:
model = Sequential()

In [913]:
model.add(GRU(input_shape=(None,1), return_sequences= True, dropout=0.2, units=3))


In [902]:
model.add(TimeDistributed(Dense(3), input_shape=()))
model.add(Activation("relu"))

In [903]:
model.compile(loss = "mse", optimizer = "sgd", metrics=['accuracy'])

In [904]:
HIDDEN_SIZE=256
BATCH_SIZE=192
NUM_EPOCHS=10000

In [905]:
model.fit(indextrain1, targettrain1, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)

Epoch 1/10000
65/65 [==============================] - 3s - loss: 4478.7905 - acc: 0.1692
Epoch 2/10000
65/65 [==============================] - 0s - loss: 4442.8174 - acc: 0.1538
Epoch 3/10000
65/65 [==============================] - 0s - loss: 4408.5469 - acc: 0.4000
Epoch 4/10000
65/65 [==============================] - 0s - loss: 4354.4604 - acc: 0.3538
Epoch 5/10000
65/65 [==============================] - 0s - loss: 4232.5044 - acc: 0.2462
Epoch 6/10000
65/65 [==============================] - 0s - loss: 4115.2861 - acc: 0.4615
Epoch 7/10000
65/65 [==============================] - 0s - loss: 4008.1584 - acc: 0.4615
Epoch 8/10000
65/65 [==============================] - 0s - loss: 3855.0635 - acc: 0.4462
Epoch 9/10000
65/65 [==============================] - 0s - loss: 3725.1763 - acc: 0.4615
Epoch 10/10000
65/65 [==============================] - 0s - loss: 3614.9199 - acc: 0.4462
Epoch 11/10000
65/65 [==============================] - 0s - loss: 3497.4333 - acc: 0.4615
Epoch 12

65/65 [==============================] - 0s - loss: 85.2018 - acc: 0.4615
Epoch 92/10000
65/65 [==============================] - 0s - loss: 76.1860 - acc: 0.4615
Epoch 93/10000
65/65 [==============================] - 0s - loss: 82.6540 - acc: 0.4615
Epoch 94/10000
65/65 [==============================] - 0s - loss: 76.8192 - acc: 0.4615
Epoch 95/10000
65/65 [==============================] - 0s - loss: 74.1006 - acc: 0.4615
Epoch 96/10000
65/65 [==============================] - 0s - loss: 67.1862 - acc: 0.4615
Epoch 97/10000
65/65 [==============================] - 0s - loss: 64.6953 - acc: 0.4615
Epoch 98/10000
65/65 [==============================] - 0s - loss: 72.4013 - acc: 0.4615
Epoch 99/10000
65/65 [==============================] - 0s - loss: 65.1311 - acc: 0.4615
Epoch 100/10000
65/65 [==============================] - 0s - loss: 59.0174 - acc: 0.4615
Epoch 101/10000
65/65 [==============================] - 0s - loss: 61.6364 - acc: 0.4615
Epoch 102/10000
65/65 [===========

65/65 [==============================] - 0s - loss: 26.3269 - acc: 0.4615
Epoch 183/10000
65/65 [==============================] - 0s - loss: 26.8193 - acc: 0.4615
Epoch 184/10000
65/65 [==============================] - 0s - loss: 27.9192 - acc: 0.4615
Epoch 185/10000
65/65 [==============================] - 0s - loss: 26.2778 - acc: 0.4615
Epoch 186/10000
65/65 [==============================] - 0s - loss: 27.5427 - acc: 0.4615
Epoch 187/10000
65/65 [==============================] - 0s - loss: 26.9335 - acc: 0.4615
Epoch 188/10000
65/65 [==============================] - 0s - loss: 26.7235 - acc: 0.4615
Epoch 189/10000
65/65 [==============================] - 0s - loss: 28.5975 - acc: 0.4615
Epoch 190/10000
65/65 [==============================] - 0s - loss: 26.2125 - acc: 0.4615
Epoch 191/10000
65/65 [==============================] - 0s - loss: 26.6065 - acc: 0.4615
Epoch 192/10000
65/65 [==============================] - 0s - loss: 26.1930 - acc: 0.4615
Epoch 193/10000
65/65 [===

65/65 [==============================] - 0s - loss: 25.9375 - acc: 0.4615
Epoch 274/10000
65/65 [==============================] - 0s - loss: 26.0219 - acc: 0.4615
Epoch 275/10000
65/65 [==============================] - 0s - loss: 26.5680 - acc: 0.4615
Epoch 276/10000
65/65 [==============================] - 0s - loss: 25.2782 - acc: 0.4615
Epoch 277/10000
65/65 [==============================] - 0s - loss: 26.3391 - acc: 0.4615
Epoch 278/10000
65/65 [==============================] - 0s - loss: 26.0388 - acc: 0.4615
Epoch 279/10000
65/65 [==============================] - 0s - loss: 26.7254 - acc: 0.4615
Epoch 280/10000
65/65 [==============================] - 0s - loss: 27.0778 - acc: 0.4615
Epoch 281/10000
65/65 [==============================] - 0s - loss: 26.0400 - acc: 0.4615
Epoch 282/10000
65/65 [==============================] - 0s - loss: 27.5363 - acc: 0.4615
Epoch 283/10000
65/65 [==============================] - 0s - loss: 26.0389 - acc: 0.4615
Epoch 284/10000
65/65 [===

65/65 [==============================] - 0s - loss: 25.9968 - acc: 0.4615
Epoch 365/10000
65/65 [==============================] - 0s - loss: 26.9895 - acc: 0.4615
Epoch 366/10000
65/65 [==============================] - 0s - loss: 25.9513 - acc: 0.4615
Epoch 367/10000
65/65 [==============================] - 0s - loss: 26.1864 - acc: 0.4615
Epoch 368/10000
65/65 [==============================] - 0s - loss: 26.3677 - acc: 0.4615
Epoch 369/10000
65/65 [==============================] - 0s - loss: 26.0545 - acc: 0.4615
Epoch 370/10000
65/65 [==============================] - 0s - loss: 26.2407 - acc: 0.4615
Epoch 371/10000
65/65 [==============================] - 0s - loss: 26.0404 - acc: 0.4615
Epoch 372/10000
65/65 [==============================] - 0s - loss: 25.8294 - acc: 0.4615
Epoch 373/10000
65/65 [==============================] - 0s - loss: 26.0402 - acc: 0.4615
Epoch 374/10000
65/65 [==============================] - 0s - loss: 26.0176 - acc: 0.4615
Epoch 375/10000
65/65 [===

...

65/65 [==============================] - 0s - loss: 26.0115 - acc: 0.4615
Epoch 4062/10000
65/65 [==============================] - 0s - loss: 26.0873 - acc: 0.4615
Epoch 4063/10000
65/65 [==============================] - 0s - loss: 25.9037 - acc: 0.4615
Epoch 4064/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4065/10000
65/65 [==============================] - 0s - loss: 26.0289 - acc: 0.4615
Epoch 4066/10000
65/65 [==============================] - 0s - loss: 26.0223 - acc: 0.4615
Epoch 4067/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4068/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4069/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4070/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4071/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4072/10000

65/65 [==============================] - 0s - loss: 26.0410 - acc: 0.4615
Epoch 4152/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4153/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4154/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4155/10000
65/65 [==============================] - 0s - loss: 26.0409 - acc: 0.4615
Epoch 4156/10000
65/65 [==============================] - 0s - loss: 26.0409 - acc: 0.4615
Epoch 4157/10000
65/65 [==============================] - 0s - loss: 25.9921 - acc: 0.4615
Epoch 4158/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4159/10000
65/65 [==============================] - 0s - loss: 26.0405 - acc: 0.4615
Epoch 4160/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4161/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4162/10000

65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4242/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4243/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4244/10000
65/65 [==============================] - 0s - loss: 26.1486 - acc: 0.4615
Epoch 4245/10000
65/65 [==============================] - 0s - loss: 26.0426 - acc: 0.4615
Epoch 4246/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4247/10000
65/65 [==============================] - 0s - loss: 26.0380 - acc: 0.4615
Epoch 4248/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4249/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4250/10000
65/65 [==============================] - 0s - loss: 26.0409 - acc: 0.4615
Epoch 4251/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4252/10000

65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4332/10000
65/65 [==============================] - 0s - loss: 26.0548 - acc: 0.4615
Epoch 4333/10000
65/65 [==============================] - 0s - loss: 26.0917 - acc: 0.4615
Epoch 4334/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4335/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4336/10000
65/65 [==============================] - 0s - loss: 26.0382 - acc: 0.4615
Epoch 4337/10000
65/65 [==============================] - 0s - loss: 26.1570 - acc: 0.4615
Epoch 4338/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4339/10000
65/65 [==============================] - 0s - loss: 26.0610 - acc: 0.4615
Epoch 4340/10000
65/65 [==============================] - 0s - loss: 26.0380 - acc: 0.4615
Epoch 4341/10000
65/65 [==============================] - 0s - loss: 26.0378 - acc: 0.4615
Epoch 4342/10000

KeyboardInterrupt: 

In [910]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_144 (GRU)                (None, None, 3)           63        
_________________________________________________________________
time_distributed_18 (TimeDis (None, None, 3)           12        
_________________________________________________________________
activation_84 (Activation)   (None, None, 3)           0         
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________
None


In [911]:
indextest.shape

(56,)

In [ ]:
indextest1=indextest.reshape()

TypeError: Error when checking model : data should be a Numpy array, or list/dict of Numpy arrays. Found: 67...